<a href="https://colab.research.google.com/github/p25-c4/kurly_project/blob/main/0320_SG_%EC%97%B0%EA%B4%80%EA%B7%9C%EC%B9%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 연관규칙

In [ ]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 전처리 : 항목 값에 대한 index 생성
from mlxtend.preprocessing import TransactionEncoder
# 지지도 계산
from mlxtend.frequent_patterns import apriori
# 연관 규칙
from mlxtend.frequent_patterns import association_rules

In [ ]:
kurly = pd.read_csv("kurly.csv")
kurly.head()

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT,Delay_YN,ItemLargeCode,ItemLargeName,ItemMiddleCode,...,Date,Year,Month,Day,Hour,CleanedItemName,TimeBetweenOrders,Combined,Weekday,WeekdayName
0,U10001,U10001-O2021-1001,2021-12-03 20:59:27,L1-M12-S0070-1113,17030,2021-12-04 03:26:00,0,L1,가공식품,M12,...,2021-12-03,2021,12,3,20,이츠웰 후르츠칵테일 과일통조림 3k 슬라이스,3 days 09:06:40,"2030s, 여성",4,Friday
1,U10001,U10001-O2021-1001,2021-12-03 20:59:27,L1-M15-S0140-1247,7680,2021-12-04 03:26:00,0,L1,가공식품,M15,...,2021-12-03,2021,12,3,20,베지가든 수제교자만두 매콤한 김치맛 720g 360gx2개입,3 days 09:06:40,"2030s, 여성",4,Friday
2,U10001,U10001-O2021-1001,2021-12-03 20:59:27,L1-M23-S0580-1047,12420,2021-12-04 03:26:00,0,L1,가공식품,M23,...,2021-12-03,2021,12,3,20,매일유업 매일 상하치즈 피자용 모짜렐라 슈레드치즈 200gx5봉,3 days 09:06:40,"2030s, 여성",4,Friday
3,U10001,U10001-O2021-1001,2021-12-03 20:59:27,L4-M12-S0640-1010,6250,2021-12-04 03:26:00,0,L4,신선식품,M12,...,2021-12-03,2021,12,3,20,계양구 배달 싱싱한 방울토마토 750g 작전 과일나라,3 days 09:06:40,"2030s, 여성",4,Friday
4,U10001,U10001-O2021-1001,2021-12-03 20:59:27,L4-M17-S0130-1018,14450,2021-12-04 03:26:00,0,L4,신선식품,M17,...,2021-12-03,2021,12,3,20,낙지호롱 900g 20 25개 낙지꼬치구이 냉동낙지 양념소스70gx1팩 제공,3 days 09:06:40,"2030s, 여성",4,Friday


In [ ]:
kurly.columns

Index(['idUser', 'idOrder', 'OrderDT', 'ItemCode', 'Price', 'DeliveryDT',
       'Delay_YN', 'ItemLargeCode', 'ItemLargeName', 'ItemMiddleCode',
       'ItemMiddleName', 'ItemSmallCode', 'ItemSmallName', 'ItemName',
       'Gender', 'Age', 'FamilyCount', 'MemberYN', 'AgeGroup', 'Date', 'Year',
       'Month', 'Day', 'Hour', 'CleanedItemName', 'TimeBetweenOrders',
       'Combined', 'Weekday', 'WeekdayName'],
      dtype='object')

In [ ]:
# 연관분석을 위한 df
kurly_2030_male = kurly.loc[kurly['Combined']=='2030s, 남성',['idOrder','ItemSmallName']]
kurly_2030_female = kurly.loc[kurly['Combined']=='2030s, 여성',['idOrder','ItemSmallName']]
kurly_4060_male = kurly.loc[kurly['Combined']=='4060s, 남성',['idOrder','ItemSmallName']]
kurly_4060_female = kurly.loc[kurly['Combined']=='4060s, 여성',['idOrder','ItemSmallName']]


kurly_2030_female.shape

(406741, 2)

## 2030 여성

In [ ]:
# 고객id 리스트 생성
id = list(set(kurly_2030_female['idOrder']))
id.sort()

# 고객id별 상품 매칭
list_asso = []
for i in id:
    tmp_list = list(kurly_2030_female[kurly_2030_female['idOrder']==i]['ItemSmallName'])
    tmp_list.sort()
    list_asso.append(tmp_list)

list_asso[:1]

[['과일', '김치', '낙지', '냉동', '전복', '치즈', '토마']]

In [ ]:
# item 기준을 ㅗ포함 여부에 따라 True/False 반환
enc = TransactionEncoder()
kurly_enc = enc.fit_transform(X=list_asso)
df_asso = pd.DataFrame(kurly_enc, columns = enc.columns_)
df_asso


,갈치,계란,고구,고등,고사,과일,국수,김,김치,깻잎,...,콩나,키위,탄산,토마,파,파스,파프,풋고,호박,혼합
0,False,False,False,False,False,True,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70530,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
70531,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
70532,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,True,False,True
70533,False,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False


In [ ]:
# 규제 선택 최소 지지도
v_min_support = 0.5

df_freq = apriori(df_asso, min_support = 0.01, verbose=-1, use_colnames=True)
df_freq.sort_values(by='support', ascending=False).round(3)

Processing 400 combinations | Sampling itemset size 4 3


,support,itemsets
11,0.329,(냉동)
28,0.277,(사과)
2,0.238,(고구)
48,0.228,(토마)
41,0.195,(즉석)
...,...,...
501,0.010,"(고구, 냉동, 상추)"
433,0.010,"(즉석, 쇠고)"
497,0.010,"(고구, 냉동, 고등)"
317,0.010,"(땅콩, 토마)"


In [ ]:
df_asso_rule_2030_female = association_rules(df_freq, metric='lift', min_threshold=1)
df_asso_rule_2030_female

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(계란),(냉동),0.049564,0.329028,0.016914,0.341247,1.037137,0.000606,1.018549,0.037674
1,(냉동),(계란),0.329028,0.049564,0.016914,0.051405,1.037137,0.000606,1.001940,0.053366
2,(고구),(김),0.237868,0.170993,0.041568,0.174753,1.021986,0.000894,1.004556,0.028228
3,(김),(고구),0.170993,0.237868,0.041568,0.243098,1.021986,0.000894,1.006910,0.025951
4,(고구),(냉동),0.237868,0.329028,0.079464,0.334068,1.015319,0.001199,1.007569,0.019797
...,...,...,...,...,...,...,...,...,...,...
419,(호박),"(즉석, 냉동)",0.175544,0.057135,0.010378,0.059118,1.034713,0.000348,1.002108,0.040692
420,"(냉동, 커피)",(토마),0.050982,0.228426,0.011980,0.234983,1.028708,0.000334,1.008572,0.029406
421,(토마),"(냉동, 커피)",0.228426,0.050982,0.011980,0.052445,1.028708,0.000334,1.001545,0.036169
422,"(호박, 토마)",(냉동),0.033203,0.329028,0.010959,0.330060,1.003135,0.000034,1.001540,0.003233


In [ ]:
#### 결과물 정제하기 ####
# dataframe으로 저장
df_asso_rule_2030_female.reset_index(inplace=True, drop=True)
# csv 파일로 결과 저장
df_asso_rule_2030_female.to_csv("2030_female_association.csv", index=False, encoding='cp949')
# dataframe으로 불러오기
df_2030_female = pd.read_csv("2030_female_association.csv", encoding='cp949')
# 특수문자 제거
df_2030_female['antecedents'] = df_2030_female['antecedents'].str[11:-2].str.replace("'", "")
df_2030_female['consequents'] = df_2030_female['consequents'].str[11:-2].str.replace("'", "")
df_2030_female

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.049564,0.329028,0.016914,0.341247,1.037137,0.000606,1.018549,0.037674
1,냉동,계란,0.329028,0.049564,0.016914,0.051405,1.037137,0.000606,1.001940,0.053366
2,고구,김,0.237868,0.170993,0.041568,0.174753,1.021986,0.000894,1.004556,0.028228
3,김,고구,0.170993,0.237868,0.041568,0.243098,1.021986,0.000894,1.006910,0.025951
4,고구,냉동,0.237868,0.329028,0.079464,0.334068,1.015319,0.001199,1.007569,0.019797
...,...,...,...,...,...,...,...,...,...,...
419,호박,"즉석, 냉동",0.175544,0.057135,0.010378,0.059118,1.034713,0.000348,1.002108,0.040692
420,"냉동, 커피",토마,0.050982,0.228426,0.011980,0.234983,1.028708,0.000334,1.008572,0.029406
421,토마,"냉동, 커피",0.228426,0.050982,0.011980,0.052445,1.028708,0.000334,1.001545,0.036169
422,"호박, 토마",냉동,0.033203,0.329028,0.010959,0.330060,1.003135,0.000034,1.001540,0.003233


In [ ]:
# 다시 csv파일로 저장
df_2030_female.to_csv("2030_female_association.csv", index=False, encoding='cp949')

## 2030 남성

In [ ]:
# 고객id 리스트 생성
id = list(set(kurly_2030_male['idOrder']))
id.sort()

# 고객id별 상품 매칭
list_asso = []
for i in id:
    tmp_list = list(kurly_2030_male[kurly_2030_male['idOrder']==i]['ItemSmallName'])
    tmp_list.sort()
    list_asso.append(tmp_list)

list_asso[:1]

[['딸기', '딸기', '라면', '미역', '소시', '수박', '수박', '파스']]

In [ ]:
# item 기준을 ㅗ포함 여부에 따라 True/False 반환
enc = TransactionEncoder()
kurly_enc = enc.fit_transform(X=list_asso)
df_asso = pd.DataFrame(kurly_enc, columns = enc.columns_)
df_asso


,갈치,계란,고구,고등,고사,과일,국수,김,김치,깻잎,...,콩나,키위,탄산,토마,파,파스,파프,풋고,호박,혼합
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52965,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,True
52966,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
52967,False,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
52968,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# 규제 선택 최소 지지도
v_min_support = 0.5

df_freq = apriori(df_asso, min_support = 0.01, verbose=-1, max_len=4, use_colnames=True)
df_freq.sort_values(by='support', ascending=False).round(3)

Processing 380 combinations | Sampling itemset size 4 3


,support,itemsets
11,0.330,(냉동)
28,0.277,(사과)
2,0.240,(고구)
48,0.223,(토마)
41,0.197,(즉석)
...,...,...
512,0.010,"(생선, 냉동, 사과)"
481,0.010,"(키위, 혼합)"
354,0.010,"(키위, 밤)"
414,0.010,"(오징, 생선)"


In [ ]:
df_asso_rule_2030_male = association_rules(df_freq, metric='lift', min_threshold=1)
df_asso_rule_2030_male

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(계란),(냉동),0.047820,0.330187,0.015839,0.331228,1.003152,0.000050,1.001556,0.003300
1,(냉동),(계란),0.330187,0.047820,0.015839,0.047970,1.003152,0.000050,1.000158,0.004692
2,(고구),(과일),0.240476,0.130923,0.031980,0.132988,1.015771,0.000497,1.002381,0.020442
3,(과일),(고구),0.130923,0.240476,0.031980,0.244268,1.015771,0.000497,1.005018,0.017865
4,(고구),(국수),0.240476,0.055107,0.013366,0.055582,1.008621,0.000114,1.000503,0.011253
...,...,...,...,...,...,...,...,...,...,...
399,(토마),"(냉동, 커피)",0.222975,0.049953,0.011856,0.053171,1.064420,0.000718,1.003399,0.077889
400,"(호박, 토마)",(냉동),0.032075,0.330187,0.011440,0.356680,1.080238,0.000850,1.041182,0.076739
401,(냉동),"(호박, 토마)",0.330187,0.032075,0.011440,0.034648,1.080238,0.000850,1.002666,0.110894
402,"(토마, 사과)",(즉석),0.051501,0.196904,0.010591,0.205645,1.044393,0.000450,1.011004,0.044814


In [ ]:
#### 결과물 정제하기 ####
# dataframe으로 저장
df_asso_rule_2030_male.reset_index(inplace=True, drop=True)
# csv 파일로 결과 저장
df_asso_rule_2030_male.to_csv("2030_male_association.csv", index=False, encoding='cp949')
# dataframe으로 불러오기
df_2030_male = pd.read_csv("2030_male_association.csv", encoding='cp949')
# 특수문자 제거
df_2030_male['antecedents'] = df_2030_male['antecedents'].str[11:-2].str.replace("'", "")
df_2030_male['consequents'] = df_2030_male['consequents'].str[11:-2].str.replace("'", "")
df_2030_male

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.047820,0.330187,0.015839,0.331228,1.003152,0.000050,1.001556,0.003300
1,냉동,계란,0.330187,0.047820,0.015839,0.047970,1.003152,0.000050,1.000158,0.004692
2,고구,과일,0.240476,0.130923,0.031980,0.132988,1.015771,0.000497,1.002381,0.020442
3,과일,고구,0.130923,0.240476,0.031980,0.244268,1.015771,0.000497,1.005018,0.017865
4,고구,국수,0.240476,0.055107,0.013366,0.055582,1.008621,0.000114,1.000503,0.011253
...,...,...,...,...,...,...,...,...,...,...
399,토마,"냉동, 커피",0.222975,0.049953,0.011856,0.053171,1.064420,0.000718,1.003399,0.077889
400,"호박, 토마",냉동,0.032075,0.330187,0.011440,0.356680,1.080238,0.000850,1.041182,0.076739
401,냉동,"호박, 토마",0.330187,0.032075,0.011440,0.034648,1.080238,0.000850,1.002666,0.110894
402,"토마, 사과",즉석,0.051501,0.196904,0.010591,0.205645,1.044393,0.000450,1.011004,0.044814


In [ ]:
# 다시 csv파일로 저장
df_2030_male.to_csv("2030_male_association.csv", index=False, encoding='cp949')

## 4060 여성

In [ ]:
# 고객id 리스트 생성
id = list(set(kurly_4060_female['idOrder']))
id.sort()

# 고객id별 상품 매칭
list_asso = []
for i in id:
    tmp_list = list(kurly_4060_female[kurly_4060_female['idOrder']==i]['ItemSmallName'])
    tmp_list.sort()
    list_asso.append(tmp_list)

list_asso[:1]

[['고등', '고사', '국수', '국수', '도라', '도라', '사과']]

In [ ]:
# item 기준을 ㅗ포함 여부에 따라 True/False 반환
enc = TransactionEncoder()
kurly_enc = enc.fit_transform(X=list_asso)
df_asso = pd.DataFrame(kurly_enc, columns = enc.columns_)
df_asso


,갈치,계란,고구,고등,고사,과일,국수,김,김치,깻잎,...,콩나,키위,탄산,토마,파,파스,파프,풋고,호박,혼합
0,False,False,False,True,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36241,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36242,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
36243,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
36244,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# 규제 선택 최소 지지도
v_min_support = 0.5

df_freq = apriori(df_asso, min_support = 0.01, verbose=-1, max_len=4, use_colnames=True)
df_freq.sort_values(by='support', ascending=False).round(3)

Processing 312 combinations | Sampling itemset size 4 3


,support,itemsets
11,0.326,(냉동)
28,0.277,(사과)
2,0.239,(고구)
48,0.225,(토마)
41,0.194,(즉석)
...,...,...
372,0.010,"(블루, 파)"
343,0.010,"(미역, 블루)"
504,0.010,"(냉동, 토마, 고등)"
85,0.010,"(고구, 아몬)"


In [ ]:
df_asso_rule_4060_female = association_rules(df_freq, metric='lift', min_threshold=1)
df_asso_rule_4060_female

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(계란),(냉동),0.048778,0.325746,0.016691,0.342195,1.050494,0.000802,1.025005,0.050532
1,(냉동),(계란),0.325746,0.048778,0.016691,0.051241,1.050494,0.000802,1.002596,0.071289
2,(고구),(과일),0.239199,0.132097,0.032417,0.135525,1.025946,0.000820,1.003965,0.033242
3,(과일),(고구),0.132097,0.239199,0.032417,0.245405,1.025946,0.000820,1.008225,0.029140
4,(고구),(국수),0.239199,0.056310,0.013491,0.056401,1.001629,0.000022,1.000097,0.002138
...,...,...,...,...,...,...,...,...,...,...
383,(냉동),"(토마, 사과)",0.325746,0.050461,0.017354,0.053273,1.055741,0.000916,1.002971,0.078306
384,"(호박, 사과)",(냉동),0.041411,0.325746,0.013574,0.327781,1.006248,0.000084,1.003028,0.006477
385,(냉동),"(호박, 사과)",0.325746,0.041411,0.013574,0.041670,1.006248,0.000084,1.000270,0.009209
386,"(냉동, 커피)",(토마),0.050571,0.224659,0.011725,0.231860,1.032053,0.000364,1.009375,0.032712


In [ ]:
#### 결과물 정제하기 ####
# dataframe으로 저장
df_asso_rule_4060_female.reset_index(inplace=True, drop=True)
# csv 파일로 결과 저장
df_asso_rule_4060_female.to_csv("4060_female_association.csv", index=False, encoding='cp949')
# dataframe으로 불러오기
df_4060_female = pd.read_csv("4060_female_association.csv", encoding='cp949')
# 특수문자 제거
df_4060_female['antecedents'] = df_4060_female['antecedents'].str[11:-2].str.replace("'", "")
df_4060_female['consequents'] = df_4060_female['consequents'].str[11:-2].str.replace("'", "")
df_4060_female

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.048778,0.325746,0.016691,0.342195,1.050494,0.000802,1.025005,0.050532
1,냉동,계란,0.325746,0.048778,0.016691,0.051241,1.050494,0.000802,1.002596,0.071289
2,고구,과일,0.239199,0.132097,0.032417,0.135525,1.025946,0.000820,1.003965,0.033242
3,과일,고구,0.132097,0.239199,0.032417,0.245405,1.025946,0.000820,1.008225,0.029140
4,고구,국수,0.239199,0.056310,0.013491,0.056401,1.001629,0.000022,1.000097,0.002138
...,...,...,...,...,...,...,...,...,...,...
383,냉동,"토마, 사과",0.325746,0.050461,0.017354,0.053273,1.055741,0.000916,1.002971,0.078306
384,"호박, 사과",냉동,0.041411,0.325746,0.013574,0.327781,1.006248,0.000084,1.003028,0.006477
385,냉동,"호박, 사과",0.325746,0.041411,0.013574,0.041670,1.006248,0.000084,1.000270,0.009209
386,"냉동, 커피",토마,0.050571,0.224659,0.011725,0.231860,1.032053,0.000364,1.009375,0.032712


In [ ]:
# 다시 csv파일로 저장
df_4060_female.to_csv("4060_female_association.csv", index=False, encoding='cp949')

## 4060 남성

In [ ]:
# 고객id 리스트 생성
id = list(set(kurly_4060_male['idOrder']))
id.sort()

# 고객id별 상품 매칭
list_asso = []
for i in id:
    tmp_list = list(kurly_4060_male[kurly_4060_male['idOrder']==i]['ItemSmallName'])
    tmp_list.sort()
    list_asso.append(tmp_list)

list_asso[:1]

[['과일', '김치', '낙지', '냉동', '전복', '치즈', '토마']]

In [ ]:
# item 기준을 ㅗ포함 여부에 따라 True/False 반환
enc = TransactionEncoder()
kurly_enc = enc.fit_transform(X=list_asso)
df_asso = pd.DataFrame(kurly_enc, columns = enc.columns_)
df_asso


,갈치,계란,고구,고등,고사,과일,국수,김,김치,깻잎,...,콩나,키위,탄산,토마,파,파스,파프,풋고,호박,혼합
0,False,False,False,True,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36241,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36242,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
36243,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
36244,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# 규제 선택 최소 지지도
v_min_support = 0.5

df_freq = apriori(df_asso, min_support = 0.01, verbose=-1, max_len=4, use_colnames=True)
df_freq.sort_values(by='support', ascending=False).round(3)

Processing 312 combinations | Sampling itemset size 4 3


,support,itemsets
11,0.326,(냉동)
28,0.277,(사과)
2,0.239,(고구)
48,0.225,(토마)
41,0.194,(즉석)
...,...,...
372,0.010,"(블루, 파)"
343,0.010,"(미역, 블루)"
504,0.010,"(냉동, 토마, 고등)"
85,0.010,"(고구, 아몬)"


In [ ]:
df_asso_rule_4060_male = association_rules(df_freq, metric='lift', min_threshold=1)
df_asso_rule_4060_male

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(계란),(냉동),0.048778,0.325746,0.016691,0.342195,1.050494,0.000802,1.025005,0.050532
1,(냉동),(계란),0.325746,0.048778,0.016691,0.051241,1.050494,0.000802,1.002596,0.071289
2,(고구),(과일),0.239199,0.132097,0.032417,0.135525,1.025946,0.000820,1.003965,0.033242
3,(과일),(고구),0.132097,0.239199,0.032417,0.245405,1.025946,0.000820,1.008225,0.029140
4,(고구),(국수),0.239199,0.056310,0.013491,0.056401,1.001629,0.000022,1.000097,0.002138
...,...,...,...,...,...,...,...,...,...,...
383,(냉동),"(토마, 사과)",0.325746,0.050461,0.017354,0.053273,1.055741,0.000916,1.002971,0.078306
384,"(호박, 사과)",(냉동),0.041411,0.325746,0.013574,0.327781,1.006248,0.000084,1.003028,0.006477
385,(냉동),"(호박, 사과)",0.325746,0.041411,0.013574,0.041670,1.006248,0.000084,1.000270,0.009209
386,"(냉동, 커피)",(토마),0.050571,0.224659,0.011725,0.231860,1.032053,0.000364,1.009375,0.032712


In [ ]:
#### 결과물 정제하기 ####
# dataframe으로 저장
df_asso_rule_4060_male.reset_index(inplace=True, drop=True)
# csv 파일로 결과 저장
df_asso_rule_4060_male.to_csv("4060_male_association.csv", index=False, encoding='cp949')
# dataframe으로 불러오기
df_4060_male = pd.read_csv("4060_male_association.csv", encoding='cp949')
# 특수문자 제거
df_4060_male['antecedents'] = df_4060_male['antecedents'].str[11:-2].str.replace("'", "")
df_4060_male['consequents'] = df_4060_male['consequents'].str[11:-2].str.replace("'", "")
df_4060_male

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.048778,0.325746,0.016691,0.342195,1.050494,0.000802,1.025005,0.050532
1,냉동,계란,0.325746,0.048778,0.016691,0.051241,1.050494,0.000802,1.002596,0.071289
2,고구,과일,0.239199,0.132097,0.032417,0.135525,1.025946,0.000820,1.003965,0.033242
3,과일,고구,0.132097,0.239199,0.032417,0.245405,1.025946,0.000820,1.008225,0.029140
4,고구,국수,0.239199,0.056310,0.013491,0.056401,1.001629,0.000022,1.000097,0.002138
...,...,...,...,...,...,...,...,...,...,...
383,냉동,"토마, 사과",0.325746,0.050461,0.017354,0.053273,1.055741,0.000916,1.002971,0.078306
384,"호박, 사과",냉동,0.041411,0.325746,0.013574,0.327781,1.006248,0.000084,1.003028,0.006477
385,냉동,"호박, 사과",0.325746,0.041411,0.013574,0.041670,1.006248,0.000084,1.000270,0.009209
386,"냉동, 커피",토마,0.050571,0.224659,0.011725,0.231860,1.032053,0.000364,1.009375,0.032712


In [ ]:
# 다시 csv파일로 저장
df_4060_male.to_csv("4060_male_association.csv", index=False, encoding='cp949')

## 결과 확인

In [ ]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)

df_4060_male.sort_values(by=['antecedents','confidence'], ascending=[True, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.048778,0.325746,0.016691,0.342195,1.050494,8.023109e-04,1.025005,0.050532
8,고구,냉동,0.239199,0.325746,0.079595,0.332757,1.021521,1.676866e-03,1.010506,0.027691
32,고구,커피,0.239199,0.172322,0.041329,0.172780,1.002653,1.093705e-04,1.000553,0.003478
6,고구,김,0.239199,0.169895,0.040639,0.169896,1.000009,3.790611e-07,1.000002,0.000012
2,고구,과일,0.239199,0.132097,0.032417,0.135525,1.025946,8.198451e-04,1.003965,0.033242
16,고구,라면,0.239199,0.118937,0.029714,0.124221,1.044428,1.263972e-03,1.006034,0.055913
30,고구,잼,0.239199,0.112675,0.027700,0.115802,1.027754,7.480016e-04,1.003537,0.035494
28,고구,스낵,0.239199,0.108315,0.026927,0.112572,1.039299,1.018195e-03,1.004797,0.049702
22,고구,미역,0.239199,0.094135,0.023644,0.098847,1.050057,1.127122e-03,1.005229,0.062658
18,고구,마른,0.239199,0.094686,0.022982,0.096078,1.014702,3.329937e-04,1.001540,0.019045


In [ ]:
df_4060_female.sort_values(by=['antecedents','confidence'], ascending=[True, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.048778,0.325746,0.016691,0.342195,1.050494,8.023109e-04,1.025005,0.050532
8,고구,냉동,0.239199,0.325746,0.079595,0.332757,1.021521,1.676866e-03,1.010506,0.027691
32,고구,커피,0.239199,0.172322,0.041329,0.172780,1.002653,1.093705e-04,1.000553,0.003478
6,고구,김,0.239199,0.169895,0.040639,0.169896,1.000009,3.790611e-07,1.000002,0.000012
2,고구,과일,0.239199,0.132097,0.032417,0.135525,1.025946,8.198451e-04,1.003965,0.033242
16,고구,라면,0.239199,0.118937,0.029714,0.124221,1.044428,1.263972e-03,1.006034,0.055913
30,고구,잼,0.239199,0.112675,0.027700,0.115802,1.027754,7.480016e-04,1.003537,0.035494
28,고구,스낵,0.239199,0.108315,0.026927,0.112572,1.039299,1.018195e-03,1.004797,0.049702
22,고구,미역,0.239199,0.094135,0.023644,0.098847,1.050057,1.127122e-03,1.005229,0.062658
18,고구,마른,0.239199,0.094686,0.022982,0.096078,1.014702,3.329937e-04,1.001540,0.019045


In [ ]:
df_2030_female.sort_values(by=['antecedents','confidence'], ascending=[True, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.049564,0.329028,0.016914,0.341247,1.037137,0.000606,1.018549,0.037674
4,고구,냉동,0.237868,0.329028,0.079464,0.334068,1.015319,0.001199,1.007569,0.019797
30,고구,즉석,0.237868,0.194627,0.047608,0.200143,1.028343,0.001312,1.006897,0.036164
34,고구,커피,0.237868,0.174084,0.042135,0.177137,1.017537,0.000726,1.003710,0.022614
2,고구,김,0.237868,0.170993,0.041568,0.174753,1.021986,0.000894,1.004556,0.028228
20,고구,생선,0.237868,0.123244,0.029659,0.124687,1.011711,0.000343,1.001649,0.015188
10,고구,라면,0.237868,0.119444,0.028766,0.120932,1.012457,0.000354,1.001693,0.016144
28,고구,잼,0.237868,0.110002,0.027263,0.114614,1.041929,0.001097,1.005209,0.052801
24,고구,스낵,0.237868,0.108329,0.026767,0.112528,1.038763,0.000999,1.004732,0.048963
16,고구,미역,0.237868,0.096817,0.023534,0.098939,1.021917,0.000505,1.002355,0.028140


In [ ]:
df_2030_male.sort_values(by=['antecedents','confidence'], ascending=[True, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,계란,냉동,0.047820,0.330187,0.015839,0.331228,1.003152,0.000050,1.001556,0.003300
8,고구,냉동,0.240476,0.330187,0.081688,0.339692,1.028788,0.002286,1.014395,0.036842
6,고구,김,0.240476,0.172362,0.042779,0.177893,1.032091,0.001330,1.006728,0.040937
2,고구,과일,0.240476,0.130923,0.031980,0.132988,1.015771,0.000497,1.002381,0.020442
22,고구,생선,0.240476,0.121408,0.029677,0.123410,1.016489,0.000481,1.002284,0.021358
14,고구,라면,0.240476,0.116896,0.028979,0.120506,1.030875,0.000868,1.004104,0.039433
30,고구,잼,0.240476,0.110459,0.027053,0.112498,1.018462,0.000490,1.002298,0.023867
26,고구,스낵,0.240476,0.106211,0.026883,0.111791,1.052541,0.001342,1.006283,0.065723
20,고구,미역,0.240476,0.096073,0.023504,0.097739,1.017339,0.000401,1.001846,0.022439
16,고구,마른,0.240476,0.090882,0.022786,0.094756,1.042629,0.000932,1.004280,0.053831
